# Movie Recomendation System

In [ ]:
import pandas as pd
import numpy as np
from functools import partialmethod
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer


import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = (stopwords.words('english'))

pd.DataFrame.head = partialmethod(pd.DataFrame.head, n=1)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
from IPython.display import clear_output
import json

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
credits_df=pd.read_csv("tmdb_5000_credits.csv")
movies_df=pd.read_csv("tmdb_5000_movies.csv")

# Info

In [ ]:
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
credits_df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [ ]:
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [ ]:
movies_df.merge(credits_df,how='inner', left_on='id', right_on='movie_id').shape[0]==movies_df.shape[0]

True

Observations -- The Datasets can be mapped by unique ID without Data Loss
common columns = title

In [ ]:
mov_cred_df=movies_df.merge(credits_df.drop(columns='title'),how='inner', left_on='id', right_on='movie_id')

In [ ]:
mov_cred_df_copy=mov_cred_df.copy()

In [ ]:
(mov_cred_df.isnull().sum()/mov_cred_df.shape[0]).sort_values().tail(6)

production_companies    0.000000
release_date            0.000208
runtime                 0.000416
overview                0.000625
tagline                 0.175724
homepage                0.643556
dtype: float64

Null Values: 64% in Home Page and 17 % in Tag Line
- receive the null information from TMDB API to fill the data set

In [ ]:
mov_cred_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
## Considering the columns that play an important role in our recommendation system

importance=pd.DataFrame()
for i in mov_cred_df.columns:
  print(" ")
  print(f"Columns {i}")
  total_unique_values = mov_cred_df[i].nunique()
  print(f"example of row {mov_cred_df[i][0]}")
  print(f"Value counts of the columns {((mov_cred_df[i].value_counts())/mov_cred_df.shape[0]).sort_values().tail(10)}")
  print(f"null values in columns {mov_cred_df[i].isnull().sum()}")
  user_input=int(input("Importance 0 or 1  - "))
  clear_output()
  row = pd.DataFrame({'column':[i], 'total_unique_values':[total_unique_values], 'is_required':[user_input]})
  importance = pd.concat([importance,row])

"""
important_columns = list(importance[importance['is_required']==1]['column'])
"""
important_columns = ['genres', 'id', 'overview', 
'popularity', 'title', 'vote_average',
'vote_count', 'cast', 'keywords', 'popularity', 'popularity',
'crew']

In [ ]:
mov_cred_df=mov_cred_df_copy.copy()

In [ ]:
mov_cred_df=mov_cred_df[important_columns]

In [ ]:
mov_cred_df.head()

,genres,id,overview,popularity,title,vote_average,vote_count,cast,keywords,popularity,popularity,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"In the 22nd century, a paraplegic Marine is di...",150.437577,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",150.437577,150.437577,"[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
mov_cred_df.isnull().sum().sort_values().tail(2)

crew        0
overview    3
dtype: int64

In [ ]:
mov_cred_df.dropna(inplace=True)

In [ ]:
mov_cred_df.duplicated().sum()

0

[Text Columns] --> genres, overview, title, cast, keywords.
- Transformations:
-- Create Tags from all text columns. 
-- Cluster on the basis of Tags



# Text Transformations

In [ ]:
text_columns = ['id', 'overview','title','cast', 'keywords']

In [ ]:
## Problems in Data
#--> keywords, genres and cast are in json format
# Some of the json values are empty
mov_cred_df.drop(index=(mov_cred_df[mov_cred_df['genres']=='[]'].index),inplace=True)

In [ ]:
(pd.read_json(mov_cred_df['genres'][0])).head()

,id,name
0,28,Action


In [ ]:
(pd.read_json(mov_cred_df['keywords'][0])).head()

,id,name
0,1463,culture clash


In [ ]:
(pd.read_json(mov_cred_df['cast'][0])).head()

,cast_id,character,credit_id,gender,id,name,order
0,242,Jake Sully,5602a8a7c3a3685532001c9a,2,65731,Sam Worthington,0


In [ ]:
(pd.read_json(mov_cred_df['crew'][0])).head()

,credit_id,department,gender,id,job,name
0,52fe48009251416c750aca23,Editing,0,1721,Editor,Stephen E. Rivkin


Converting the json columns into list

In [ ]:
def remove_jsons(df):
  """
  This function converts json columns to lists of required values from the json
  
  --Json --pd.read_json()--> df -.values--> array using lambda function on required columns
  -- in Crew we need name for job=Director and for rest we need names column
  --there are some empyt jsons in the columns which are handled by logic the no of rows in df loaded by json<>0 
  --Restrict cast to just three elements
  """
  for i in ['genres','keywords','cast']:
   df[i]=df[i].apply(lambda x: (pd.read_json(x))['name'].values if (pd.read_json(x)).shape[0]!=0 else [])
  #for crew we will restrict job to director
  def get_director(bd):
    return(bd[bd['job']=='Director']['name'].values)
  df['crew']=df['crew'].apply(lambda x: get_director(pd.read_json(x)) if (pd.read_json(x)).shape[0]!=0 else [])
  # restricting cast to 3
  df['cast']=df['cast'].apply(lambda x:x[:3])
  return(df)

In [ ]:
mov_cred_df = remove_jsons(mov_cred_df)

In [ ]:
mov_cred_df.head()

,genres,id,overview,popularity,title,vote_average,vote_count,cast,keywords,popularity,popularity,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"In the 22nd century, a paraplegic Marine is di...",150.437577,Avatar,7.2,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[culture clash, future, space war, space colon...",150.437577,150.437577,[James Cameron]



  - Cleaning Columns
  -- Remove Blank spaces in all names
  -- Convert Overview to List
  -- concatenate all list
  -- convert to lower
  -- stemming
  -- convert the concatenated list into one description string


In [ ]:
def list_transformation(A):
  B=[]
  for i in A:
    i=i.lower()
    i=i.replace(" ","")
    i = ps.stem(i)
    if i in stop_words:
      pass
    else:
      B.append(i)

  return(B)


In [ ]:
mov_cred_df['overview']=mov_cred_df['overview'].apply(lambda x:x.split(" "))

In [ ]:
for i in ['genres','keywords','cast','crew','overview']:
  mov_cred_df[i]=mov_cred_df[i].apply(lambda x:list_transformation(x))

In [ ]:
mov_cred_df['all_tags']=mov_cred_df['genres']+mov_cred_df['overview']+mov_cred_df['cast']+mov_cred_df['keywords']+mov_cred_df['crew']

In [ ]:
mov_cred_df['all_tags']=mov_cred_df['all_tags'].apply(lambda x:' '.join(x))

In [ ]:
text_df=mov_cred_df[['id', 'title', 'all_tags']]

In [ ]:
text_df.set_index('id',inplace=True)

# Vectorization

In [ ]:
cv=CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(text_df['all_tags']).toarray()

In [ ]:
vectors.shape

(4772, 5000)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity.shape

(4772, 4772)

Int64Index([ 19995,    285, 206647,  49026,  49529,    559,  38757,  99861,
               767, 209112,
            ...
             36095, 182291, 286939, 124606,  14337,  67238,   9367,  72766,
            231617,  25975],
           dtype='int64', name='id', length=4772)

In [ ]:
similarity_df = pd.DataFrame(similarity)
similarity_df.columns = text_df.index
similarity_df.set_index(text_df.index,inplace=True)
similarity_df

In [ ]:
similarity_df.head()

id     19995     285       206647    49026     49529     559       38757   \
id                                                                          
19995     1.0  0.083462  0.086031  0.073472  0.189299  0.108389  0.040242   

id       99861     767      209112    1452     10764     58        57201   \
id                                                                          
19995  0.146735  0.059235  0.09673  0.102598  0.09465  0.090371  0.045431   

id       49521     2454      24428     1865      41154     122917    1930    \
id                                                                            
19995  0.128247  0.062828  0.078947  0.140848  0.094933  0.083081  0.058038   

id       20662     57158     2268      254       597       271110    44833   \
id                                                                            
19995  0.111283  0.066227  0.087407  0.053338  0.051016  0.153897  0.193381   

id       135397    37724     558       68721     12155     36668   62211   \
id                                                                          
19995  0.116543  0.065033  0.066848  0.159071  0.084478  0.097333     0.0   

id      8373      91314     68728     102382    20526     49013     44912   \
id                                                                           
19995  0.09934  0.171191  0.078947  0.081111  0.082261  0.076948  0.165637   

id     10193     534       168259    72190     127585    54138     81005   \
id                                                                          
19995     0.0  0.090862  0.033826  0.081752  0.139104  0.196722  0.084478   

id       64682     9543      68726     38356     217      105864    62177   \
id                                                                           
19995  0.059235  0.112956  0.080484  0.146735  0.042974  0.03122  0.072098   

id       188927    10681     5174      14161     17979   76757     258489  \
id                                                                          
19995  0.138343  0.024183  0.067488  0.072548  0.018368  0.2227  0.083771   

id       411       246655    155       14160     15512     1726      44826   \
id                                                                            
19995  0.081111  0.030124  0.019529  0.022496  0.162221  0.120495  0.026579   

id       8487      1735      297761    2698      137113    9804      14869   \
id                                                                            
19995  0.076004  0.073367  0.137102  0.033113  0.202465  0.103905  0.071755   

id       150540    278927    10138     58595     102651    119450    79698   \
id                                                                            
19995  0.014395  0.050669  0.062311  0.050465  0.110208  0.076004  0.179969   

id       64686     100402    10192     158852    177572    82690     5255    \
id                                                                            
19995  0.123613  0.114127  0.070987  0.111283  0.090371  0.057807  0.061314   

id       47933     10191     296       118340    157336    27205     315011  \
id                                                                            
19995  0.166924  0.062311  0.071755  0.187317  0.140488  0.076948  0.069523   

id       49051     9799      4922      49538     131634    27022     503     \
id                                                                            
19995  0.075094  0.026669  0.067651  0.076948  0.124622  0.079872  0.055641   

id       241259    810       68735     87101     10140     676       1858    \
id                                                                            
19995  0.103757  0.062311  0.061806  0.145095  0.064889  0.044992  0.150188   

id       1966      675       674       8960      6479      118       2062    \
id                                                                            
19995  0.076948  0.043355  0.045883  0.043748  0.047836  0.059235  0.0

In [ ]:
def recommend_movie(title):
  index = text_df[text_df['title']==title].index.values[0]
  distances = pd.DataFrame(similarity_df.loc[index])
  suggestions_index = distances.sort_values(by=index,ascending=False).index[:5]
  suggestions_name = text_df[text_df.index.isin(suggestions_index)]['title'].values
  return(suggestions_name)

In [ ]:
recommend_movie('Spectre')

array(['Spectre', 'Quantum of Solace', 'Skyfall', 'Never Say Never Again',
       'From Russia with Love'], dtype=object)

In [ ]:
recommend_movie('Avatar')

array(['Avatar', 'Independence Day', 'Aliens vs Predator: Requiem',
       'Aliens', 'Falcon Rising'], dtype=object)

In [ ]:
recommend_movie('Cavite')

array(['The Diving Bell and the Butterfly', 'Flying By',
       "Should've Been Romeo", '1982', 'Cavite'], dtype=object)

In [ ]:
recommend_movie("Pirates of the Caribbean: At World\'s End")

array(["Pirates of the Caribbean: At World's End",
       "Pirates of the Caribbean: Dead Man's Chest",
       'Pirates of the Caribbean: On Stranger Tides',
       'Pirates of the Caribbean: The Curse of the Black Pearl',
       'Life of Pi'], dtype=object)

In [ ]:
text_df.to_pickle("movies_list.pkl")
similarity_df.to_pickle("similarity_df.pkl")